In [9]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import pprint

pp = pprint.PrettyPrinter(indent=4)
out = JupyterOut.unit_square( )
item = "item"

# Multi-Dimensionality
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P23.jpg" style="width: 600px; display: inline;">

## Multiple Paths to Higher Dimensions

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P21.jpg" style="width: 200px; display: inline;"><img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P22.jpg" style="width: 200px; display: inline;">

### Nests in Python

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P15.jpg" style="width: 200px; display: inline;">

In [13]:
"""
A Grid of Points
In contrast with other languages such as C#, Python Lists are not typed and 
not sized at construction. To create a List of a given size, we must fill 
it with elements.
"""
pt_grid_python = []
for ni in range(4):
    row = []
    for mi in range(3): row.append(Point(ni,mi))
    pt_grid_python.append(row)
    
pp.pprint(pt_grid_python)

[   [pt[0,0,0], pt[0,1,0], pt[0,2,0]],
    [pt[1,0,0], pt[1,1,0], pt[1,2,0]],
    [pt[2,0,0], pt[2,1,0], pt[2,2,0]],
    [pt[3,0,0], pt[3,1,0], pt[3,2,0]]]


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P16.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P17.jpg" style="width: 200px; display: inline;">

In [14]:
"""
Nest
We define a as a non-rectangular or "jagged" array in which each row of 
objects may contain a different number of items. Sometimes termed a "list 
of lists", a nest maintains a consistent dimension, and, unlike a tree, 
does not store items at multiple levels.
"""
nest = []
nest.append( [item,item,item,item,item,item,item,item] ) # nest[0]
nest.append( [item,item,item] ) # nest[1]
nest.append( [item,item,item,item] ) # nest[2]
nest.append( [item,item,item,item,item,item,item,item] )# nest[3]
nest.append( [item,item] ) # nest[4]

pp.pprint(nest)

[   ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    ['item', 'item', 'item'],
    ['item', 'item', 'item', 'item'],
    ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    ['item', 'item']]


### Trees in Python

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P18.jpg" style="width: 600px; display: inline;">

In [15]:
"""
Tree
A tree is a multi-level collection that is non-rectangular, and that mixes 
objects and collections at a variety of levels. Unlike a nest, it is not 
possible to determine the rank of a tree.
"""
tree = []
tree.append( [item,item,item,item,item,item,item,item] )
tree.append( item )
tree.append( [item,item,item,item] )
tree.append( [item,item,item,item,item,item,item,item] )
tree.append( [ [item,item] ,item] )

pp.pprint(tree)

[   ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    'item',
    ['item', 'item', 'item', 'item'],
    ['item', 'item', 'item', 'item', 'item', 'item', 'item', 'item'],
    [['item', 'item'], 'item']]


## A Glossary of Multi-Dimensional Collections